### Imports

In [12]:
# !pip install xlrd
# !pip install plotly
# !pip install jupyter_dash

In [7]:
import pandas as pd
import numpy as np
import pickle

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from jupyter_dash import JupyterDash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import base64
import io

import json
from dash.exceptions import PreventUpdate

pd.options.mode.chained_assignment = None # default='warn'

In [8]:
!git clone https://github.com/jhochs/Fleetwise.git

Cloning into 'Fleetwise'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 56 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (56/56), done.


In [9]:
def import_subWO_pull(filename):
    df = pd.read_excel(filename, na_values='')
    return df

def WO_from_subWO(subWO):
    WO = subWO.groupby(['Work Order Id']).agg({'Asset Id':'first', 'Item Desc':'first', 'Priority Cd':'first', 'Sub Work Order State Cd':pd.Series.mode, 'Actual Labor Cost':sum, 'Actual Labor Hours':sum, 'Actual Non-Labor Cost':sum, 'Est Labor Cost':sum, 'Est Labor Hours':sum, 'Est Non-Labor Cost':sum, 'Estbd Dt/Time':min, 'Last Tran Dt/Time':max,  'Asset LIN/TAMCN':'first', 'Closed Dt':max, 'Maint Team Desc':pd.Series.mode, 'Service End Dt':max, 'Downtime days':max})
    return WO

def calculate_downtime(WO):
    # today = pd.Timestamp('today')
    today = pd.to_datetime('2021-05-05 00:00:00') #for development
    WO['Downtime days'] = (today - WO['Estbd Dt/Time']).astype('timedelta64[D]').astype(int)
    return WO

def import_MVR_pull(filename):
    MVR = pd.read_excel(filename, skiprows=11, na_values='')
    MVR.dropna(axis=1, how='all', inplace=True) #drop empty columns
    MVR.dropna(axis=0, how='all', inplace=True) #drop empty rows

    # Extract authorized and assigned and drop the rest:
    MVR_types = MVR.loc[MVR['Auth NSN'] == 'Sub Totals:']
    MVR_types.dropna(axis=1, how='all', inplace=True) #drop empty columns
    MVR_types.drop(columns=['Auth NSN', 'Asset NSN', 'AUTH MGMT', 'MGMT', 'Account', 'EQP CD', 'Auth QTY', 'Auth Use Code', 'OC', 'UC', 'Unit', 'User'], inplace=True)
    MVR_types.rename(columns={'Vehicle Type Name': 'Authorized', 'ASC': 'Assigned'}, inplace=True)

    # Reference vehicle type from original dataframe and add it to this table:
    MVR_types['Vehicle Type Name'] = MVR.loc[MVR_types.index - 1, 'Vehicle Type Name'].values
    MVR_types['MGMT'] = MVR.loc[MVR_types.index - 1, 'MGMT'].values
    MVR_types.dropna(subset=['MGMT'], inplace=True) # drop empty rows
    MVR_types.set_index('MGMT', inplace=True) # reindex on MGMT 

    # For the original dataframe, remove unneeded rows and columns:
    MVR['Asset NSN'] = pd.to_numeric(MVR['Asset NSN'], errors='coerce')
    MVR.dropna(subset=['Asset NSN'], inplace=True) # drop empty rows
    MVR.drop(MVR.columns.difference(['Asset NSN','Vehicle Type Name', 'MGMT', 'Account', 'ASC', 'Reg Number', 'Unit', 'User', 'EOL']), 1, inplace=True)
    
    # Create dict connecting reg number and unit assigned
    reg_unit_dict = pd.Series(MVR['Unit'].values, index=MVR['Reg Number'].values).to_dict()
    reg_unit_dict = {'AF'+k: v for k, v in reg_unit_dict.items()} # add AF to beginning of reg number to match sub-WO pull

    return MVR, MVR_types, reg_unit_dict

def import_AF_fleet(filename):
    # Dataframe with every vehicle owned by the AF:
    AF_fleet = pd.read_excel(filename)
    AF_fleet.dropna(axis=0, subset=['Mgmt Cd'], inplace=True) #drop empty rows
    
    # Create dictionary for MGMT code (type) and category:
    AF_fleet_types = AF_fleet.drop_duplicates(subset=['Mgmt Cd'])
    type_cat_dict = pd.Series(AF_fleet_types['VEH Cat'].values, index=AF_fleet_types['Mgmt Cd'].values).to_dict()
    
    # List of the 12 categories:
    cats = pd.unique(AF_fleet_types['VEH Cat'])
    
    return AF_fleet, type_cat_dict, cats

def import_type_cat(dict_filename, list_filename):
    with open(dict_filename, 'rb') as f:
      type_cat_dict = pickle.load(f)

    with open(list_filename, 'rb') as f:
      cats = pickle.load(f)

    return type_cat_dict, cats

In [13]:
# IMPORT FILES
subWO = import_subWO_pull('Fleetwise/2020-2021 Sub Work Order Inquiry.xls')
MVR, MVR_types, reg_unit_dict = import_MVR_pull('Fleetwise/MVR Hand Receipt 6 May 2021.xls')
type_cat_dict, cats = import_type_cat('Fleetwise/type_cat_dict.p', 'Fleetwise/cats.p')
# AF_fleet, type_cat_dict, cats = import_AF_fleet("/content/drive/MyDrive/H4D/DoD Docs/ROSARIO, SIGOURNEY A SSgt USAF AMC 60 LRSLGRV/AF Fleet_Posture.xlsx")

print('Import complete')

Import complete


In [14]:
# DECLARE FUNCTIONS FOR GENERATING DASHBOARD

def calculate_MCR(open_WO, MVR): # todo: totals = 0 validation
  # CALCULATE MC RATE BY VEH CATEGORY

  totals = []
  NMCS = []
  NMCM = []
  
  for cat in cats:
      # Sum total number of vehicles on base with this category:
      totals.append(len(MVR.loc[MVR['VEH Cat'] == cat]))
      
      # NMCS has state code AWSM:
      NMCS_WO = open_WO.loc[np.logical_and((open_WO['VEH Cat'] == cat), \
                                          (open_WO['Sub Work Order State Cd'] == 'AWSM-Apprvd-in shop awtng mtrls, wrk stop'))]
      NMCS.append(len(NMCS_WO))
      
      # NMCM has state codes AIPR, CAWI, IIPR:
      NMCM_WO = open_WO.loc[np.logical_and((open_WO['VEH Cat'] == cat), \
                                          (open_WO['Sub Work Order State Cd'].str.contains('AIPR|CAWI|IIPR', regex=True)))]
      NMCM.append(len(NMCM_WO))
      
      # QUESTION: different state codes for sub work orders?
  
  # Calculate percentage rates:
  MCR = 100 * np.divide(np.subtract(totals, np.add(NMCM, NMCS)), totals)
  MCR_overall = 100 * (1 - (np.sum(NMCM)+np.sum(NMCS)) / sum(totals) )
  NMCSR = 100 * np.divide(NMCS, totals)
  NMCMR = 100 * np.divide(NMCM, totals)

  return MCR, MCR_overall, NMCSR, NMCMR

def calculate_overview_numbers(open_WO, WO):
  # CALCULATE OVERVIEW NUMBERS (opened, closed, count by ETIC):

  # today = pd.Timestamp('today')
  today = pd.to_datetime('2021-05-04 00:00:00') # for development, since DPAS pull is not up to date
  
  week_ago = today - pd.Timedelta(value=7, unit='days')
  month_ago = today - pd.Timedelta(value=30, unit='days')
  ranges = [week_ago, month_ago]

  open = len(open_WO)
  ETIC_expired = len(open_WO[open_WO['Service End Dt'] < today])
  ETIC_1week = len(open_WO[np.logical_and((open_WO['Service End Dt'] > today), (open_WO['Service End Dt'] < today + pd.Timedelta(value=7, unit='days')))])

  opened = []
  closed = []
  for t0 in ranges:
    opened.append(len(WO[WO['Estbd Dt/Time'] > t0]))
    closed.append(len(WO[WO['Closed Dt'] > t0]))

  return open, opened, closed, ETIC_expired, ETIC_1week

def downtime_report(WO, intervals):
  # GENERATE NUMBERS FOR DOWNTIME BAR CHART:

  WO = WO.loc[WO['Sub Work Order State Cd'].str.contains('AWSM|AIPR|CAWI|IIPR', regex=True)]
  MGMTs = WO['Asset LIN/TAMCN'].unique()
  counts = np.zeros((len(MGMTs), len(intervals)))

  for i, mgmt in enumerate(MGMTs):
    # Get the open WOs for this MGMT code only:
    WO_mgmt = WO.loc[WO['Asset LIN/TAMCN'] == mgmt]
    for j in range(len(intervals)-1):
      counts[i,j] = len(WO_mgmt.loc[np.logical_and(WO_mgmt['Downtime days'] > intervals[j], WO_mgmt['Downtime days'] <= intervals[j+1])])
    # The final column is for number of vehicles with downtime greater than the highest interval:
    counts[i,j+1] = len(WO_mgmt.loc[WO_mgmt['Downtime days'] > intervals[j+1]])

  return MGMTs, counts.astype(int)

def subWO_table(subWO_df, reg_unit_dict):
  # CREATE SUBWO DATAFRAME FOR TABLE DISPLAY:

  # Delete the columns not of interest from the open sub WO dataframe:
  subWO_df_tab = subWO_df.drop(subWO_df.columns.difference(['Work Order Id', 'Sub Work Order Id', 'Asset Id', 'Sub Work Order State Cd', 'Asset LIN/TAMCN', 'Unit', 'Estbd Dt/Time', 'Maint Team Desc', 'Remarks', 'Service Performed', 'Service End Dt', 'Downtime days']), 1)

  #Drop any rows with state codes other than AWSM|AIPR|CAWI|IIPR:
  subWO_df_tab = subWO_df_tab[subWO_df_tab['Sub Work Order State Cd'].str.contains('AWSM|AIPR|CAWI|IIPR', regex=True)]

  # Add column for unit:
  subWO_df_tab.insert(loc=3, column='Unit', value=subWO_df_tab['Asset Id'].map(reg_unit_dict))

  # Convert ETIC from datetime to date:
  subWO_df_tab['ETIC'] = subWO_df_tab['Service End Dt'].dt.strftime('%Y-%m-%d') #.dt.date
  subWO_df_tab.drop(columns=['Service End Dt'], inplace=True)

  # Add column for downtime:
  # today = pd.Timestamp('today')
  today = pd.to_datetime('2021-05-04 00:00:00') #for development
  subWO_df_tab['Downtime days'] = (today - subWO_df_tab['Estbd Dt/Time']).astype('timedelta64[D]').astype(int)
  subWO_df_tab.drop(columns=['Estbd Dt/Time'], inplace=True)

  # Reorder columns:
  subWO_df_tab = subWO_df_tab[['Work Order Id', 'Asset Id', 'Asset LIN/TAMCN', 'Unit', 'ETIC', 'Downtime days', 'Sub Work Order Id', 'Sub Work Order State Cd', 'Maint Team Desc', 'Service Performed', 'Remarks']]
  
  # Sort by WO ID (with secondary sort by sub WO ID):
  subWO_df_tab.sort_values(by=['Work Order Id', 'Sub Work Order Id'], inplace=True)

  return subWO_df_tab

def create_plots(MCR, MCR_overall, NMCSR, NMCMR,  MGMTs, counts, open, opened, closed, ETIC_expired, ETIC_1week):
  # MAKE DASHBOARD PLOTS:

  # today = pd.Timestamp('today')
  today = pd.to_datetime('2021-05-04 00:00:00') #for development

  fig = make_subplots(
      rows=4, cols=6,
      row_heights=[0.16, 0.37, 0.10, 0.37],
      vertical_spacing=0.15,
      specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"},  None],
            [{"type": "bar", "colspan": 3}, None,  None, {"type": "bar", "colspan": 3},  None,  None],
            [{"colspan": 6}, None,  None,  None,  None,  None], # empty row because bar chart xlabels extend down
            [{"colspan": 6}, None,  None,  None,  None,  None]],
      subplot_titles=("Open work orders", "Opened in last month", "Closed in last month", "ETIC expired", "ETIC due in next 7 days", "MCR - Current  (Overall: " + str(MCR_overall)[0:4] + "%)", "NMCR - Current", "", "Downtime report (open work orders)"))

  # Numbers overview:
  fig.add_trace(go.Indicator(mode = "number", value = open), row=1, col=1)
  fig.add_trace(go.Indicator(mode = "number", value = opened[1]), row=1, col=2)
  fig.add_trace(go.Indicator(mode = "number", value = closed[1]), row=1, col=3)
  fig.add_trace(go.Indicator(mode = "number", value = ETIC_expired), row=1, col=4)
  fig.add_trace(go.Indicator(mode = "number", value = ETIC_1week), row=1, col=5)

  # MCR graph:
  target = 90
  fig.add_trace(go.Scatter(name='Target', x=cats, marker_color="red", y=target*np.ones(cats.shape), mode='lines', legendgroup = '1', hoverinfo='skip'), row=2, col=1)
  fig.add_trace(go.Bar(name='MCR', x=cats, y=MCR, marker_color="royalblue", legendgroup = '1', hovertemplate='<b>%{x}</b>: %{y:.2f}%'), row=2, col=1)

  # NMCR graph:
  fig.add_trace(go.Bar(name='NMCS', x=cats, y=NMCSR, marker_color="gold", legendgroup = '2', hovertemplate='<b>%{x}</b>: %{y:.2f}%'), row=2, col=4)
  fig.add_trace(go.Bar(name='NMCM', x=cats, y=NMCMR, marker_color="coral", legendgroup = '2', hovertemplate='<b>%{x}</b>: %{y:.2f}%'), row=2, col=4)
  
  # Downtime report graph:
  fig.add_trace(go.Bar(name='0-30 days', x=MGMTs, y=counts[:,0], marker_color="gold", legendgroup = '3', hovertemplate='<b>%{x}</b>: %{y:.0f}'), row=4, col=1)
  fig.add_trace(go.Bar(name='30-60 days', x=MGMTs, y=counts[:,1], marker_color="orange", legendgroup = '3', hovertemplate='<b>%{x}</b>: %{y:.0f}'), row=4, col=1)
  fig.add_trace(go.Bar(name='30-90 days', x=MGMTs, y=counts[:,2], marker_color="coral", legendgroup = '3', hovertemplate='<b>%{x}</b>: %{y:.0f}'), row=4, col=1)
  fig.add_trace(go.Bar(name='90+ days', x=MGMTs, y=counts[:,3], marker_color="orangered", legendgroup = '3', hovertemplate='<b>%{x}</b>: %{y:.0f}'), row=4, col=1)
  
  '''
  # Sub-WO table:
  fig.add_trace(
      go.Table(
          header=dict(
              values=["<b>Work Order ID</b>", "<b>Sub Work Order ID</b>", "<b>Asset ID</b>", "<b>Sub Work Order<br>State Code</b>",
                      "<b>Maint Team</b>", "<b>ETIC</b>", "<b>Service</b>"],
              font=dict(size=10),
              align="left"
          ),
          cells=dict(
              values=[open_subWO[k].tolist() for k in open_subWO.columns],
              font=dict(size=10),
              align = "left")
      ),
      row=4, col=1
  )
  '''
  fig.update_layout({"barmode":"stack"})
  fig.update_layout(height=600, title_text="<b>4th Logistics Readiness Squadron: Fleet Overview - " + today.strftime("%d %b %Y") + "</b>")
  
  return fig

def create_dashboard(subWO):
  # Calculate donwtime days:
  subWO = calculate_downtime(subWO)
  
  # Create new dataframe with only open work orders:
  open_subWO = subWO.loc[subWO['Work Order Status Cd'] == 'O-Open']

  # Create new dataframes dropping repetitive sub work orders
  open_WO = WO_from_subWO(open_subWO)
  WO = WO_from_subWO(subWO)

  # Add category columns:
  open_WO['VEH Cat'] = open_WO['Asset LIN/TAMCN'].map(type_cat_dict)
  MVR['VEH Cat'] = MVR['MGMT'].map(type_cat_dict)
  
  MCR, MCR_overall, NMCSR, NMCMR = calculate_MCR(open_WO, MVR)
  open, opened, closed, ETIC_expired, ETIC_1week = calculate_overview_numbers(open_WO, WO)
  MGMTs, counts = downtime_report(open_WO, [0, 30, 60, 90])
  open_subWO_tab = subWO_table(open_subWO, reg_unit_dict)
  fig = create_plots(MCR, MCR_overall, NMCSR, NMCMR, MGMTs, counts, open, opened, closed, ETIC_expired, ETIC_1week)

  return fig, open_subWO_tab

In [15]:
fig, open_subWO_tab = create_dashboard(subWO)

In [16]:
### Stylesheet

stylesheet = {'upload': {
                'width': '50%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '3px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
                },
              'filters': {
                'width' : '300px'
                  },
              'graph' : {
                'height': '40%', 
                'width': '100%', 
                'marginLeft': 'auto', 
                'marginRight': 'auto'
              },

              'table-cell': {
                'whiteSpace': 'normal',
                'textAlign': 'left',
                'fontSize':11, 
                'font-family':'arial',
                'padding': '4px'
              },
              'table-header': {
                'whiteSpace': 'normal',
                'fontWeight': 'bold'
              },
              'table-style': {
                'height': '50%', 
                'width': '90%',
                'overflowY': 'scroll', 
                'marginLeft': 'auto', 
                'marginRight': 'auto'
              },

              'filter-container': {
                'margin': '20px',
                'display': 'flex',
                'flex-direction': 'row',
                'justify-content': 'space-between',
              }
              }

In [17]:
# OPEN DASHBOARD IN NEW WINDOW:

app = JupyterDash(__name__)
app.layout = html.Div([
    ### Data store
    dcc.Store(id='table-data'),

    ### Upload
    html.H1("Upload DPAS Sub Work Order Pull Here"),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop OR ',
            html.A('Click to Select Files')
        ]),
        style=stylesheet['upload'], 
        multiple=True),

    html.Br(),

    ### Dashboard graphs
    html.H3('Dashboard Graphs'),
    dcc.Loading(
     id = 'dashboard-loading',
     type='circle',
     children=[
      html.Div(
      id = 'dashboard-container',
      children = [
        dcc.Graph(id='dashboard',
                  style=stylesheet['graph'])   
      ])
     ]),

    html.Div(
      id = 'dashboard-prompt',
      children = [
        html.H5("Upload data to generate graphs")
      ]
    ),

    ### Filters
    html.H3('Filters'),
    html.Div([
      html.Div(dcc.Dropdown(id='unit_filters', 
                            multi=True, 
                            options=[{'label': unit, 'value': unit} for unit in open_subWO_tab['Unit'].unique()],
                            placeholder="Units"),
                style=stylesheet['filters']),

      
      html.Div(dcc.Dropdown(id='status_filters', 
                            multi=True,
                            options=[{'label': status, 'value': status} for status in open_subWO_tab['Sub Work Order State Cd'].unique()],
                            placeholder='Vehicle status'),
                 style=stylesheet['filters']),
      
      html.Div(dcc.Dropdown(id='veh_type_filters',
                            multi=True,
                            options=[{'label': veh_type, 'value': veh_type} for veh_type in open_subWO_tab['Asset LIN/TAMCN'].unique()],
                            placeholder='Vehicle types'),
                style=stylesheet['filters'])
    ],
      style=stylesheet['filter-container']),
    
    ### Work order table
    html.H3("Work Order Table"),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in open_subWO_tab.columns #["Work Order ID", "Sub Work Order ID", "Asset ID", "Sub Work Order State Code", "Maint Team", "ETIC", "Service"]
        ],
        # data=open_subWO_tab.to_dict('records'),
        style_cell=stylesheet['table-cell'],
        style_header=stylesheet['table-header'],
        style_cell_conditional=[{
        'if': {'column_id': 'Sub Work Order State Code'},
        'leftBorder': 'rgb(30, 30, 30)',
        'color': 'white'
        }],
        style_header_conditional=[{
        'if': {'column_editable': False},
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white'
        }],
        style_table=stylesheet['table-style'],
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="none",
        export_format="csv",
        virtualization=True # makes it load faster, but for some reason this prevents text wrapping
    )
])

# Callback: Hides dashboard graphs when store is empty / no data is being uploaded
@app.callback(
    [Output('dashboard-container','style'),
     Output('dashboard-prompt', 'style')],
    [Input('dashboard','figure')]
)
def graph_hide(fig):
    if fig is None:
        return dict(display='none'), dict(margin='10px')
    else:
        return dict(), dict(display='none')

# Callback: Triggered by file import, updates dashboard and data store with table-data
@app.callback([Output('dashboard', 'figure'),
               Output('table-data', 'data')],
              [Input('upload-data', 'contents'),
              Input('upload-data', 'filename')], prevent_initial_call=True)
def update_dashboard(list_of_contents, list_of_filename):
    if list_of_contents is not None:
      contents = list_of_contents[0]
      filename = list_of_filename[0]
      content_type, content_string = contents.split(',')
      decoded = base64.b64decode(content_string)

      try:
          if 'csv' in filename:
              # Assume that the user uploaded a CSV file
              df = pd.read_csv(
                  io.StringIO(decoded.decode('utf-8')))
          elif 'xls' in filename:
              # Assume that the user uploaded an excel file
              df = pd.read_excel(io.BytesIO(decoded))
      except Exception as e:
          print(e)
          return html.Div([
              'There was an error processing this file.'
          ])
      fig, open_subWO_tab = create_dashboard(df)

      return fig, open_subWO_tab.to_json()


# Callback: Triggered by data store change or filters, updates data table
@app.callback(Output('datatable-interactivity', 'data'),
              [Input('unit_filters', 'value'),
               Input('status_filters', 'value'),
               Input('veh_type_filters', 'value'),
               Input('table-data', 'data')], prevent_initial_call=True)
def update_table(unit_filters, status_filters, veh_type_filters, store):
    if store:
      open_subWO_tab = pd.read_json(store)

      if unit_filters:
        open_subWO_tab = open_subWO_tab[open_subWO_tab['Unit'].isin(unit_filters)]
        
      if status_filters:
        open_subWO_tab = open_subWO_tab[open_subWO_tab['Sub Work Order State Cd'].isin(status_filters)]

      if veh_type_filters:
        open_subWO_tab = open_subWO_tab[open_subWO_tab['Asset LIN/TAMCN'].isin(veh_type_filters)]

      return open_subWO_tab.to_dict('records')
    else:
      raise PreventUpdate

# Run app and display result inline in the notebook
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
